# CAO 
***
## Author: Fionn McCarthy

## Introduction and Background
***

Below shows the libraries imported to carry out the relevant analysis

In [1]:
# Regular expressions 
import re

# Working with HTTP 
import requests as rq

# Data frames 
import pandas as pd

# Date and time library
import datetime as dt 

# For downloading from URl
import urllib.request as urlrq

### The link to the CAO points 2021 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2021)**.

In [2]:
# Retrieve the CAO points from the URl below
CAO1 = rq.get('http://www2.cao.ie/points/l8.php')

# View the above is okay, 200 means all okay. 
CAO1 

<Response [200]>

### Saving the Original Data Set 
The datetime library is usesd in order to take a back up of the data retrieved from the CAO website.The reason the datetime library is used is to back up the data each time the notebook is run as to not overwrite the data. 

In [23]:
# date and time of right now.
now = dt.datetime.now()

# Formatting now as a string.
nowstring = now.strftime('%Y%m%d_%H%M%S')

The server advises to utilize charset 'iso-8859-1' in order to decode the but from research this charset is unable to decode \x96 as it is not defined so we use 'cp1252' instead.

In [24]:
# Setting the charset to decode with 
CAO1.encoding = 'cp1252'

In [25]:
# Setting the charset to decode with 
CAO1.encoding = 'cp1252'

# Creating a file path for the data.
path1 = 'data/cao2021_' + nowstring + '.html'

# Saving the original html file before going any further. 
with open(path1, 'w') as f:
    f.write(CAO1.text)

### Regular expressions selects the lines in the we want to keep.

In [26]:
# The regular expression is compiled below 
re_expr = re.compile(r'([A-Z]{2}[0-9]{3})(.*)')

In [30]:
# This fucntion will split out the points that had either an '#' or '*' associated with them.
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
    random = ''
    if s[-1] == '*':
        random = '*'
    points = ''
    for i in s:
        if i.isdigit():
            points = points + i
    # Return an array with points, portfolio, random        
    return [points, portfolio, random]

In [27]:
# Path creation for the csv file.
path2 = 'data/cao2021_csv_' + nowstring + '.csv'

Looping through lines 

In [29]:
# Keep track of how many courses we process.
no_lines = 0

# Open and write to the cvs file
with open(path2, 'w') as f:
    # Loop through lines of the response
    for line in CAO1.iter_lines():
        # Decode using cp1252
        dline = line.decode('cp1252')
        # Match only the lines representing courses.
        if re_expr.fullmatch(dline):
            # Add one to the lines counter.
            no_lines = no_lines + 1
            # The course code, first five characters in string
            course_code = dline[:5]
            # The course title
            course_title = dline[7:57]
            # Course points split on one or more spaces after character 60
            course_points = re.split('  +', dline[60:]) 
            if len(course_points) != 2:
                course_points = course_points[:2]
            # join the fields using a comma 
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')
        
print(no_lines)

949


#### The above count of 949 courses was verified by a manual count that was carried out in visual studio code on original html file.

### The link to the CAO points 2020 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2020)**.

#### Save Original 2020 Data

In [9]:
# Creating a file path for the excel data.
path2 = 'data/cao2020_' + nowstring + '.xlsx'

In [10]:
# https://stackoverflow.com/questions/19602931/basic-http-file-downloading-and-saving-to-disk-in-python
# Save original file
urlrq.urlretrieve("http://www2.cao.ie/points/CAOPointsCharts2020.xlsx", path2) 

('data/cao2020_20211201_211941.xlsx',
 <http.client.HTTPMessage at 0x1af0fc15310>)

#### Load Spreadhseet using Pandas

In [11]:
# Download and parse the excel spreadsheet
# From examining the excel file I noted that needed to skip the first 10 trows of excel file that is loaded into dataframe
df2020 = pd.read_excel('http://www2.cao.ie/points/CAOPointsCharts2020.xlsx', skiprows = 10)

In [12]:
df2020

,CATEGORY (i.e.ISCED description),COURSE TITLE,COURSE CODE2,R1 POINTS,R1 Random *,R2 POINTS,R2 Random*,EOS,EOS Random *,EOS Mid-point,...,avp,v,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8
0,Business and administration,International Business,AC120,209,NaN,NaN,NaN,209,NaN,280,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Humanities (except languages),Liberal Arts,AC137,252,NaN,NaN,NaN,252,NaN,270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Arts,"First Year Art & Design (Common Entry,portfolio)",AD101,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Arts,Graphic Design and Moving Image Design (portfo...,AD102,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Arts,Textile & Surface Design and Jewellery & Objec...,AD103,#+matric,NaN,NaN,NaN,#+matric,NaN,#+matric,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1459,Manufacturing and processing,Manufacturing Engineering,WD208,188,NaN,NaN,NaN,188,NaN,339,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1460,Information and Communication Technologies (ICTs),Software Systems Development,WD210,279,NaN,NaN,NaN,279,NaN,337,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1461,Information and Communication Technologies (ICTs),Creative Computing,WD211,271,NaN,NaN,NaN,271,NaN,318,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1462,Personal services,Recreation and Sport Management,WD212,270,NaN,NaN,NaN,270,NaN,349,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
df2020.iloc[-1]

CATEGORY (i.e.ISCED description)          Engineering and engineering trades
COURSE TITLE                        Mechanical and Manufacturing Engineering
COURSE CODE2                                                           WD230
R1 POINTS                                                                253
R1 Random *                                                              NaN
R2 POINTS                                                                NaN
R2 Random*                                                               NaN
EOS                                                                      253
EOS Random *                                                             NaN
EOS Mid-point                                                            369
LEVEL                                                                      8
HEI                                        Waterford Institute of Technology
Test/Interview #                                                         NaN

#### Save Pandas Dataframe

In [14]:
# Creating a file path for the pandas data.
path3 = 'data/cao2020_' + nowstring + '.csv'

In [15]:
# Save pandas datafram to folder
df2020.to_csv(path3)

### The link to the CAO points 2019 can be acccessed **[here](http://www.cao.ie/index.php?page=points&p=2019)**.

#### In order to reproduce the data these are the steps I followed 
1. Download the original PDF file from the CAO website (link above). 
2. Open the original file using Microsoft Word. 
3. When open in Word, save the document as a word document. 
4. Re-save the Word document for editing.
5. Delete headers and page numbers in footer.
6. Select all in document and copy. 
7. Paste into Notepad++.
8. Remove institution name lines and blank lines.
9. In column headings, replace 'COURSE AND INSTITUTION' with 'Course'.
10. Change backticks to apostrophes. 
11. Delete tabs at end of lines: 61, 64, 169, 171, 172, 200, 335, 352, 425, 433, 700, 701, 752, 793, 830, 837, 869 and 894. 
12. Remove tab after 'Mid' first line.
13. Remove double tab on line 37.
14. Delete tabs at end of lines: 28, 107, 178, 201, 266, 370, 373, 391, 415, 437, 464, 494, 518, 535, 557, 571, 604, 625, 670, 679, 689, 785, 795




In [16]:
df2019 = pd.read_csv('data/cao2019_20211130_205230_edited.csv', sep='\t')

In [17]:
df2019

,Course Code,Course,EOS,Mid
0,AL801,Software Design with Virtual Reality and Gaming,304,328.0
1,AL802,Software Design with Cloud Computing,301,306.0
2,AL803,Software Design with Mobile Apps and Connected...,309,337.0
3,AL805,Network Management and Cloud Infrastructure,329,442.0
4,AL810,Quantity Surveying,307,349.0
...,...,...,...,...
925,WD200,Arts (options),221,296.0
926,WD210,Software Systems Development,271,329.0
927,WD211,Creative Computing,275,322.0
928,WD212,Recreation and Sport Management,274,311.0


#### Load Spreadhseet using Pandas

---

## References 